In [ ]:
import json
import urllib
import os
import csv

# 0. Preliminary step to get the dataset




Getting data from sharable google drive folder 

In [ ]:
# link to folder: https://drive.google.com/drive/folders/18TjiltRr8CFlx0aPcLsnKBr5iyeiQxWc?usp=sharing
# upload data folder to your drive root folder 'My Drive' (It is the default folder).

In [ ]:
# connect your drive to Google Colab
from google.colab import drive
drive.mount('/content/drive')
# after running this cell, open the url that displays below from your uni gmail account
# copy the code that is displayed 
# paste the code into the cell below when prompted and then press enter

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
folder_path = '/content/drive/My Drive/data_tm_workflow/'

In [ ]:
# set the output folder in the drive (the csv files with the results of the process will be downloaded there)
# first create the empty folder in the drive
output_folder = '/content/drive/My Drive/tests'

# 1. Data preparation

## 1.1. Creating data frame
A dataframe is first created to keep the documents at their initial state, and the name of each file

In [ ]:
import pandas as pd

In [ ]:
files_list = os.listdir(folder_path)

In [ ]:
#insert file names into a df
sources = pd.DataFrame(files_list, columns=['file_name'])

In [ ]:
#function to read the content of the text files
def readTxtContent(fileName):
  with open(folder_path + fileName, 'r') as file:
    return ' ' + file.read().replace('\n', ' ') + ' '

In [ ]:
# adding a column to the dataframe containing file content
sources['file_content'] = sources['file_name'].apply(lambda x: readTxtContent(x))

## 1.2. Adding columns for dates, publications and filtering dataset

### 1.2.1. dates, publications




In [ ]:
import re

In [ ]:
# function to retrieve publication ref from file name
def get_ref(file):
  ref_match = re.findall(r'(\w+\d+)_\d{4}-\d{2}-\d{2}_',file)
  return ref_match[0]

# function to retrieve date from file name
def get_date(file):
  date_match = re.findall(r'_(\d{4}-\d{2}-\d{2})_',file)
  return date_match[0]

# function to retrieve year from file name
def get_year(file):
  year_match = re.findall(r'_(\d{4})-\d{2}-\d{2}_',file)
  return year_match[0]

# function to retrieve month from file name
def get_month(file):
  month_match = re.findall(r'_\d{4}-(\d{2})-\d{2}_',file)
  return month_match[0]

# function to retrieve day from file name
def get_day(file):
  month_match = re.findall(r'_\d{4}-\d{2}-(\d{2})_',file)
  return month_match[0]

In [ ]:
sources['date'] = sources['file_name'].apply(lambda x: get_date(x))
sources['year'] = sources['file_name'].apply(lambda x: get_year(x))
sources['month'] = sources['file_name'].apply(lambda x: get_month(x))
sources['day'] = sources['file_name'].apply(lambda x: get_day(x))
sources['publication'] = sources['file_name'].apply(lambda x: get_ref(x))

In [ ]:
# add publication names

In [ ]:
def get_pub_name(pub_number):
    if (pub_number == 'sn85066408'):
        return 'L\'Italia'
    elif (pub_number == '2012271201'):
        return 'Cronaca Sovversiva'

In [ ]:
sources['publication_name'] = sources['publication'].apply(lambda x: get_pub_name(x))

### 1.2.2. Filtering

In [ ]:
import datetime
from datetime import timedelta, date

In [ ]:
# Start from 06.06.1903 and finish 01.05.1919

In [ ]:
date_ref_1 = date(1903,6,6)
date_ref_2 = date(1919,5,1)

In [ ]:
def filter_by_date(date_item,date_ref_1,date_ref_2):
    year = re.findall(r'(\d{4})-\d{2}-\d{2}',date_item)[0]
    month = re.findall(r'\d{4}-(\d{2})-\d{2}',date_item)[0]
    day = re.findall(r'\d{4}-\d{2}-(\d{2})',date_item)[0]
    file_date = date(int(year),int(month),int(day))
    if (date_ref_1 <= file_date <= date_ref_2):
        return 'included'
    else:
        return 'not included'

In [ ]:
sources['status'] = sources['date'].apply(lambda x: filter_by_date(x,date_ref_1,date_ref_2))

In [ ]:
sources

,file_name,file_content,date,year,month,day,publication,publication_name,status
0,2012271201_1904-03-26_ed-1_seq-1_ocr.txt,SOVVERSIVA' .MA Ebdomadario anarchico propaga...,1904-03-26,1904,03,26,2012271201,Cronaca Sovversiva,included
1,2012271201_1912-06-15_ed-1_seq-1_ocr.txt,ANNO Per che per chi mandano morire PREZUSE C...,1912-06-15,1912,06,15,2012271201,Cronaca Sovversiva,included
2,2012271201_1905-04-15_ed-1_seq-1_ocr.txt,"ANNO III BARRE VERMONT, SABATO APRILE Num. Sw...",1905-04-15,1905,04,15,2012271201,Cronaca Sovversiva,included
3,2012271201_1905-01-07_ed-1_seq-1_ocr.txt,"ANNO NUMERO SATUIIDAY, JANUARY BAREE, VERMONT...",1905-01-07,1905,01,07,2012271201,Cronaca Sovversiva,included
4,2012271201_1915-09-11_ed-1_seq-1_ocr.txt,"VIVA fra guerrafondai, della gente seria, che...",1915-09-11,1915,09,11,2012271201,Cronaca Sovversiva,included
...,...,...,...,...,...,...,...,...,...
766,2012271201_1904-08-13_ed-1_seq-1_ocr.txt,ANNO (UNION NUMERO GRON SOVVERSIVA Ebdomadari...,1904-08-13,1904,08,13,2012271201,Cronaca Sovversiva,included
767,2012271201_1911-12-16_ed-1_seq-1_ocr.txt,XNNO Poictò tdto noi! 'tf'Z' minatori general...,1911-12-16,1911,12,16,2012271201,Cronaca Sovversiva,included
768,2012271201_1913-02-08_ed-1_seq-1_ocr.txt,ANNO Aldamas condannato processo iniziatosi G...,1913-02-08,1913,02,08,2012271201,Cronaca Sovversiva,included
769,2012271201_1917-01-06_ed-1_seq-1_ocr.txt,Mif f'f WÈÈmm:Amy iWli Li;' moi iONfrmiv SM-:...,1917-01-06,1917,01,06,2012271201,Cronaca Sovversiva,included


In [ ]:
# variable containing the documents separately
corpus_df = sources[sources['status'] == 'included'].copy().reset_index()

## 1.2 Removing stop words, punctuation, short words

In [ ]:
! pip install nltk

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# add tokenized documents in dataframe
corpus_df['tokens'] = corpus_df['file_content'].apply(lambda x: nltk.word_tokenize(x))

In [ ]:
# possible user options:
# .isalnum() to removes tokens that include numbers
# .isalpha() to remove all tokens that contain more than letters (punctuation and numbers)
# .isdecimal() to remove tokens that contain only decimals
# .isdigit() to remove tokens that contain only digits

# add new column in df with processed tokens (here: keeping only alpha tokens longer than 3 characters + lowercasing)
corpus_df['doc_prep'] = corpus_df['tokens'].apply(lambda x: [w.lower() for w in x if (w.isalpha() and len(w) > 2 )])

In [ ]:
# link to custom stop words: https://drive.google.com/file/d/1VVfW6AKPbb7_fICOG73lEgkXmmZ6BkpC/view?usp=sharing
# Upload stop words list into Colab files before proceeding with the next cells

In [ ]:
from nltk.corpus import stopwords
ital_stopwords = stopwords.words('italian')
en_stopwords = stopwords.words('english')

In [ ]:
stop_words = pd.read_csv('stop_words.csv')

In [ ]:
stopwords = stop_words['stopword'].values.tolist()

In [ ]:
# add english stop words list to custom stopwords 
stopwords.extend(en_stopwords)

In [ ]:
# to append list of words added by user: ital_stopwords.extend(user_input)
# to remove words: ital_stopwords.remove(user_input)

In [ ]:
# add column with tokenized documents without sw
corpus_df['doc_prep_nostop'] = corpus_df['doc_prep'].apply(lambda x: [w for w in x if not w in stopwords])

In [ ]:
corpus_df['doc_prep_nostop']

0      [sovversiva, ebdomadario, anarchico, propagand...
1      [mandano, morire, prezuse, contekioni, felice,...
2      [barre, vermont, sabato, aprile, num, swhvìv, ...
3      [numero, satuiiday, january, baree, vermont, s...
4      [viva, guerrafondai, seria, persuade, tirtei, ...
                             ...                        
766    [union, numero, gron, sovversiva, ebdomadario,...
767    [xnno, poictò, tdto, minatori, campi, antracit...
768    [aldamas, condannato, processo, iniziatosi, ge...
769    [mif, wèèmm, amy, iwli, moi, ionfrmiv, mjèr, c...
770    [numero, ebdomadario, anarchico, propaganda, r...
Name: doc_prep_nostop, Length: 771, dtype: object

In [ ]:
# set the variable to use for topic modelling (if no further options are used)
corpus_model = corpus_df['doc_prep_nostop']

In [ ]:
# save the data after pre-processing in the output folder for verification of pre-processing steps
# corpus_df.to_csv(output_folder + '/corpus_df.csv')
corpus_df.to_csv(output_folder + '/corpus_df.csv')

## 1.4 Lemmatize

In [ ]:
# Lemmatization is available in multiple languages in Spacy and not in NLTK (only English)
# With Spacy, lemmatization is available for 10 languages. There's also a multi-language option that
# should be tested if additional languages are needed

In [ ]:
!pip install spacy

In [ ]:
!python3 -m spacy download it_core_news_sm

     |████████████████████████████████| 14.5 MB 1.3 MB/s eta 0:00:01
Using legacy setup.py install for it-core-news-sm, since package 'wheel' is not installed.
    Running setup.py install for it-core-news-sm ... done
✔ Download and installation successful
You can now load the model via spacy.load('it_core_news_sm')


In [ ]:
import it_core_news_sm
it_nlp = it_core_news_sm.load(disable=['tagger', 'parser', 'ner'])

In [ ]:
# lemmatization function
def lemmatize(doc):
  lemmatized_doc = []
  for w in doc:
    w_lemma = [token.lemma_ for token in it_nlp(w)]
    lemmatized_doc.append(w_lemma[0])
  return lemmatized_doc

In [ ]:
# takes a long time to run
# add column with lemmatized tokens - directly from the tokens as preprocessing has already been done
corpus_df['doc_lemmatized'] = corpus_df['doc_prep_nostop'].apply(lambda x: lemmatize(x))

In [ ]:
# variable with lemmatized tokens
lemmatized_corpus = corpus_df['doc_lemmatized']
# the lemmatized version is not used in this example

# 2. Topics with LDA

In [ ]:
#Gensim installation
! pip install gensim

In [ ]:
import gensim
from gensim.test.utils import common_corpus, common_dictionary
from gensim import corpora, models
from gensim.models.wrappers import LdaMallet

## 2.1 Preliminary steps to run LDA



### 2.1.1 Creating the dictionary, optional filtering of extreme values

In [ ]:
# if you use lemmatized version replace "corpus_model" by lemmatized_corpus
id2word = corpora.Dictionary(corpus_model)

In [ ]:
id2word.filter_extremes(no_below=5)

### 2.1.2 Bag of words

In [ ]:
corpus = [id2word.doc2bow(text) for text in corpus_model]

## 2.2 LDA

In [ ]:
from gensim.models.coherencemodel import CoherenceModel

In [ ]:
# set parameters for running the model below
topics_nr = []
coherence_values_gensim = []
for num_topics in range(3, 20):
  # gensim default parameters
  # model = gensim.models.ldamulticore.LdaMulticore(corpus, id2word=id2word, num_topics=num_topics)
  # preset1
  # model = gensim.models.ldamulticore.LdaMulticore(corpus, id2word=id2word, num_topics=num_topics,chunksize=2000, iterations = 400, passes = 20)
  # preset2
  model = gensim.models.ldamulticore.LdaMulticore(corpus, id2word=id2word, num_topics=num_topics,chunksize=1000, iterations = 200, passes = 10)
  topic_print = model.print_topics(num_words=30)
  df_topic_print = pd.DataFrame(topic_print, columns=['topic_id','words'])
  # change the file name to have a different file for each set of parameters
  csv_filenametxt = "{}/preset_2_{}_topics.csv"
  csv_filename = csv_filenametxt.format(output_folder,num_topics)
  df_topic_print.to_csv(csv_filename)
  coherencemodel = CoherenceModel(model=model, texts=corpus_model, dictionary=id2word, coherence='c_v')
  coherence_value = coherencemodel.get_coherence()
  coherence_values_gensim.append(coherence_value)
  topics_nr.append(str(num_topics))

In [ ]:
df_coherence = pd.DataFrame(topics_nr, columns=['topic_id'])

In [ ]:
df_coherence['topic_coherence'] = coherence_values_gensim

In [ ]:
# change the name of the file depending on the preset
df_coherence.to_csv(output_folder + '/preset_2_coherence_values.csv')